In [ ]:
import networkx as nx
import pandas as pd
import json
import csv
import datetime as dt
import time
from datetime import date
import tweepy
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from google.colab import drive
from networkx.algorithms.community.centrality import girvan_newman

**DATA PREPARATION**

In [ ]:
dataset= pd.read_csv('https://github.com/nanahsn/AJS/blob/main/dataset_vaksin.csv?raw=true')
training_set = dataset.iloc[:,1:2].values

In [ ]:
print(dataset)

           id             date  \
0    1.35E+18  1/10/2021 16:14   
1    1.35E+18  1/10/2021 16:33   
2    1.35E+18   1/11/2021 0:24   
3    1.35E+18   1/11/2021 0:30   
4    1.35E+18   1/11/2021 1:00   
..        ...              ...   
301  1.35E+18  1/12/2021 16:14   
302  1.35E+18  1/12/2021 16:14   
303  1.35E+18  1/12/2021 16:36   
304  1.35E+18  1/12/2021 16:39   
305  1.35E+18  1/12/2021 17:16   

                                                  text  \
0    Tak Kenal maka Tak Kebal. Vaksin menjadi bukti...   
1    Sistem satu data mendukung Vaksinasi di Indone...   
2    Apa Perbedaan Vaksin China Sinovac dan Sinopha...   
3    Dua Lagi Obat yang Dapat Selamatkan Nyawa Pasi...   
4    Vaksin buatan Tiongkok kembali dipertanyakan, ...   
..                                                 ...   
301  Vaksinasi untuk Indonesia bebas dari pandemi. ...   
302  Vaksinasi untuk Indonesia bebas dari pandemi. ...   
303  Jadilah Pelopor VACCINE Covid-19 #Vaksin #Vaks...   
304  Vaksin

In [ ]:
dataset.head(10)

,id,date,text,hashtags,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,source
0,1.35E+18,1/10/2021 16:14,Tak Kenal maka Tak Kebal. Vaksin menjadi bukti...,NaN,DiskominfoKotaBogor,Kota Bogor - Jawa Barat,Akun twitter resmi Dinas Komunikasi dan Inform...,5/6/2014 2:08,4617.0,203.0,38,Twitter for iPhone
1,1.35E+18,1/10/2021 16:33,Sistem satu data mendukung Vaksinasi di Indone...,"['Vaksin', 'Covid19', 'CoronaVirus', 'Kesehatan']",Indah Nur Hidayah,NaN,NaN,11/25/2020 3:24,10.0,22.0,38,Twitter Web App
2,1.35E+18,1/11/2021 0:24,Apa Perbedaan Vaksin China Sinovac dan Sinopha...,"['Vaksin', 'Perusahaanfarmasi', 'Virus', 'Farm...",BaBe - Baca Berita,Indonesia,Aku resmi BaBe - Aplikasi berita pintar yang m...,NaN,NaN,NaN,NaN,NaN
3,1.35E+18,1/11/2021 0:30,Dua Lagi Obat yang Dapat Selamatkan Nyawa Pasi...,"['Infus', 'Virus', 'Wabah', 'Radang', 'Vaksin']",BaBe - Baca Berita,Indonesia,Aku resmi BaBe - Aplikasi berita pintar yang m...,NaN,NaN,NaN,NaN,NaN
4,1.35E+18,1/11/2021 1:00,"Vaksin buatan Tiongkok kembali dipertanyakan, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.35E+18,1/11/2021 1:07,"Kabar Duka, Dokter Pribadi Paus Fransiskus Men...","['Vatikan', 'PausFransiskus', 'Vaksinasi', 'It...",BaBe - Baca Berita,Indonesia,Aku resmi BaBe - Aplikasi berita pintar yang m...,NaN,NaN,NaN,NaN,NaN
6,1.35E+18,1/11/2021 1:41,Masker bekas rawan tertular virus. Pakai maske...,NaN,Randy Aldino,NaN,Pemuda melek politik,3/16/2018 15:05,1776.0,1959.0,7795,Twitter for Android
7,1.35E+18,1/11/2021 1:59,"27 wakil supervisor kab/kota sudah terlatih, 6...",NaN,BP2D Jabar | #75TahunJabar,Jl. Kawaluyaan Indah Raya No 6,Badan Penelitian dan Pengembangan Daerah Provi...,NaN,NaN,NaN,NaN,NaN
8,1.35E+18,1/11/2021 2:04,Mengenal Vaksin Covid-19 !!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.35E+18,1/11/2021 2:07,Top News Koran Rakyat Merdeka,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Extracting Mentions

In [ ]:
dataset.drop_duplicates(keep=False, inplace=True)
dataset

,id,date,text,hashtags,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,source
0,1.35E+18,1/10/2021 16:14,Tak Kenal maka Tak Kebal. Vaksin menjadi bukti...,NaN,DiskominfoKotaBogor,Kota Bogor - Jawa Barat,Akun twitter resmi Dinas Komunikasi dan Inform...,5/6/2014 2:08,4617.0,203.0,38,Twitter for iPhone
1,1.35E+18,1/10/2021 16:33,Sistem satu data mendukung Vaksinasi di Indone...,"['Vaksin', 'Covid19', 'CoronaVirus', 'Kesehatan']",Indah Nur Hidayah,NaN,NaN,11/25/2020 3:24,10.0,22.0,38,Twitter Web App
2,1.35E+18,1/11/2021 0:24,Apa Perbedaan Vaksin China Sinovac dan Sinopha...,"['Vaksin', 'Perusahaanfarmasi', 'Virus', 'Farm...",BaBe - Baca Berita,Indonesia,Aku resmi BaBe - Aplikasi berita pintar yang m...,NaN,NaN,NaN,NaN,NaN
3,1.35E+18,1/11/2021 0:30,Dua Lagi Obat yang Dapat Selamatkan Nyawa Pasi...,"['Infus', 'Virus', 'Wabah', 'Radang', 'Vaksin']",BaBe - Baca Berita,Indonesia,Aku resmi BaBe - Aplikasi berita pintar yang m...,NaN,NaN,NaN,NaN,NaN
4,1.35E+18,1/11/2021 1:00,"Vaksin buatan Tiongkok kembali dipertanyakan, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
288,1.35E+18,1/12/2021 16:01,... SIAP VAKSIN .. #vaksin #siapvaksincovid19 ...,"['vaksin', 'siapvaksincovid19']",Eka Sulistiyana,Blitar Gunungkidul,"Lecturer, Entrepreneur",NaN,NaN,NaN,NaN,NaN
296,1.35E+18,1/12/2021 16:08,Vaksinasi untuk Indonesia bebas dari pandemi. ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,1.35E+18,1/12/2021 16:36,Jadilah Pelopor VACCINE Covid-19 #Vaksin #Vaks...,"['Vaksin', 'VaksinSinovac', 'vaksinuntukkita',...",rOelex,"-3.680613,128.209951. Ambon",Profesional Internal Auditor. Freelance Fotogr...,3/8/2011 9:52,59.0,155.0,524,Twitter for iPhone
304,1.35E+18,1/12/2021 16:39,"Vaksin, Bisnis atau Solusi Pandemi? https://t....","['Vaksin', 'Bisnis', 'Pandemi', 'Sinovac']",SUARAISLAM.ID,"DKI Jakarta, Indonesia",Akun Twitter Suara Islam Online,NaN,NaN,NaN,NaN,NaN


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293 entries, 0 to 305
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                293 non-null    object 
 1   date              292 non-null    object 
 2   text              292 non-null    object 
 3   hashtags          113 non-null    object 
 4   user_name         199 non-null    object 
 5   user_location     116 non-null    object 
 6   user_description  168 non-null    object 
 7   user_created      149 non-null    object 
 8   user_followers    149 non-null    float64
 9   user_friends      140 non-null    float64
 10  user_favourites   148 non-null    object 
 11  source            149 non-null    object 
dtypes: float64(2), object(10)
memory usage: 29.8+ KB


In [ ]:
# replace data
replaced=0
for i in range(526,39):
    for j in range(0,280):
        if (dataset.iloc[i,j]=='?'):
            replaced =replaced+1
print(replaced)
dataset = dataset.replace('?', np.NaN)

0


In [ ]:
#dimensi data audit risk
dataset.shape

(293, 12)

In [ ]:
#mencari value kosong
dataset.isnull().sum()

id                    0
date                  1
text                  1
hashtags            180
user_name            94
user_location       177
user_description    126
user_created        144
user_followers      144
user_friends        153
user_favourites     145
source              144
dtype: int64

In [ ]:
# merubah nilai NaN menjadi nilai rata-rata untuk mengatasi missing value
dataset = dataset.fillna(dataset.median())
dataset

<ipython-input-925-26e338572624>:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  dataset = dataset.fillna(dataset.median())


,id,date,text,hashtags,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,source
0,1.35E+18,1/10/2021 16:14,Tak Kenal maka Tak Kebal. Vaksin menjadi bukti...,NaN,DiskominfoKotaBogor,Kota Bogor - Jawa Barat,Akun twitter resmi Dinas Komunikasi dan Inform...,5/6/2014 2:08,4617.0,203.0,38,Twitter for iPhone
1,1.35E+18,1/10/2021 16:33,Sistem satu data mendukung Vaksinasi di Indone...,"['Vaksin', 'Covid19', 'CoronaVirus', 'Kesehatan']",Indah Nur Hidayah,NaN,NaN,11/25/2020 3:24,10.0,22.0,38,Twitter Web App
2,1.35E+18,1/11/2021 0:24,Apa Perbedaan Vaksin China Sinovac dan Sinopha...,"['Vaksin', 'Perusahaanfarmasi', 'Virus', 'Farm...",BaBe - Baca Berita,Indonesia,Aku resmi BaBe - Aplikasi berita pintar yang m...,NaN,57.0,48.0,NaN,NaN
3,1.35E+18,1/11/2021 0:30,Dua Lagi Obat yang Dapat Selamatkan Nyawa Pasi...,"['Infus', 'Virus', 'Wabah', 'Radang', 'Vaksin']",BaBe - Baca Berita,Indonesia,Aku resmi BaBe - Aplikasi berita pintar yang m...,NaN,57.0,48.0,NaN,NaN
4,1.35E+18,1/11/2021 1:00,"Vaksin buatan Tiongkok kembali dipertanyakan, ...",NaN,NaN,NaN,NaN,NaN,57.0,48.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
288,1.35E+18,1/12/2021 16:01,... SIAP VAKSIN .. #vaksin #siapvaksincovid19 ...,"['vaksin', 'siapvaksincovid19']",Eka Sulistiyana,Blitar Gunungkidul,"Lecturer, Entrepreneur",NaN,57.0,48.0,NaN,NaN
296,1.35E+18,1/12/2021 16:08,Vaksinasi untuk Indonesia bebas dari pandemi. ...,NaN,NaN,NaN,NaN,NaN,57.0,48.0,NaN,NaN
303,1.35E+18,1/12/2021 16:36,Jadilah Pelopor VACCINE Covid-19 #Vaksin #Vaks...,"['Vaksin', 'VaksinSinovac', 'vaksinuntukkita',...",rOelex,"-3.680613,128.209951. Ambon",Profesional Internal Auditor. Freelance Fotogr...,3/8/2011 9:52,59.0,155.0,524,Twitter for iPhone
304,1.35E+18,1/12/2021 16:39,"Vaksin, Bisnis atau Solusi Pandemi? https://t....","['Vaksin', 'Bisnis', 'Pandemi', 'Sinovac']",SUARAISLAM.ID,"DKI Jakarta, Indonesia",Akun Twitter Suara Islam Online,NaN,57.0,48.0,NaN,NaN


In [ ]:
#mencari value kosong
dataset.isnull().sum()

id                    0
date                  1
text                  1
hashtags            180
user_name            94
user_location       177
user_description    126
user_created        144
user_followers        0
user_friends          0
user_favourites     145
source              144
dtype: int64

In [ ]:
data_string = dataset.to_string(index=False)

In [ ]:
# # .astype(str) and .astype('str') keep the column as object.
# dataset['user_name'] = dataset['user_name'].astype(str)
# dataset.dtypes


In [ ]:
#country    object
dataset['user_name'] = dataset['user_name'].astype(pd.StringDtype())
dataset['hashtags'] = dataset['hashtags'].astype(pd.StringDtype())
dataset['text'] = dataset['text'].astype(pd.StringDtype())
dataset['date'] = dataset['date'].astype(pd.StringDtype())

In [ ]:
#info data
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293 entries, 0 to 305
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                293 non-null    object 
 1   date              292 non-null    string 
 2   text              292 non-null    string 
 3   hashtags          113 non-null    string 
 4   user_name         199 non-null    string 
 5   user_location     116 non-null    object 
 6   user_description  167 non-null    object 
 7   user_created      149 non-null    object 
 8   user_followers    293 non-null    float64
 9   user_friends      293 non-null    float64
 10  user_favourites   148 non-null    object 
 11  source            149 non-null    object 
dtypes: float64(2), object(6), string(4)
memory usage: 29.8+ KB


In [ ]:
tweetfile = 'https://github.com/nanahsn/AJS/blob/main/vaksin.csv?raw=true'
tweets = pd.read_csv(tweetfile)

In [ ]:
tweets['text'] = tweets['text'].str.strip('"')
dat = tweets[~tweets['text'].astype(str).str.startswith('RT @')]
dat = dat[~dat['text'].astype(str).str.startswith('rt @')]
#Sometimes people type "RT" at the beginning of a Tweet to indicate that they are re-posting someone else's content
#Removed Retweets
dat.head()

,id,date,text,hashtags,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,source
0,"1,35E+18",10/01/2021 16:14,Tak Kenal maka Tak Kebal. Vaksin menjadi bukti...,NaN,DiskominfoKotaBogor,Kota Bogor - Jawa Barat,Akun twitter resmi Dinas Komunikasi dan Inform...,06/05/2014 2:08,4617,203,38,Twitter for iPhone
1,"1,35E+18",10/01/2021 16:33,Sistem satu data mendukung Vaksinasi di Indone...,"['Vaksin', 'Covid19', 'CoronaVirus', 'Kesehatan']",Indah Nur Hidayah,NaN,NaN,25/11/2020 3:24,10,22,38,Twitter Web App
2,"1,35E+18",11/01/2021 0:24,Apa Perbedaan Vaksin China Sinovac dan Sinopha...,"['Vaksin', 'Perusahaanfarmasi', 'Virus', 'Farm...",BaBe - Baca Berita,Indonesia,Aku resmi BaBe - Aplikasi berita pintar yang m...,NaN,NaN,NaN,NaN,NaN
3,"1,35E+18",11/01/2021 0:30,Dua Lagi Obat yang Dapat Selamatkan Nyawa Pasi...,"['Infus', 'Virus', 'Wabah', 'Radang', 'Vaksin']",BaBe - Baca Berita,Indonesia,Aku resmi BaBe - Aplikasi berita pintar yang m...,NaN,NaN,NaN,NaN,NaN
4,"1,35E+18",11/01/2021 1:00,"Vaksin buatan Tiongkok kembali dipertanyakan, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dat.loc[dat['user_name']=='DiskominfoKotaBogor']

,id,date,text,hashtags,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,source
0,"1,35E+18",10/01/2021 16:14,Tak Kenal maka Tak Kebal. Vaksin menjadi bukti...,NaN,DiskominfoKotaBogor,Kota Bogor - Jawa Barat,Akun twitter resmi Dinas Komunikasi dan Inform...,06/05/2014 2:08,4617,203,38,Twitter for iPhone


In [ ]:
def betweenness_centrality(G, k=None, normalized=True, weight=None,
						endpoints=False, seed=None):
	r"""Compute the shortest-path betweenness centrality for nodes.

	Betweenness centrality of a node $v$ is the sum of the
	fraction of all-pairs shortest paths that pass through $v$

	.. math::

	c_B(v) =\sum_{s,t \in V} \frac{\sigma(s, t|v)}{\sigma(s, t)}

	where $V$ is the set of nodes, $\sigma(s, t)$ is the number of
	shortest $(s, t)$-paths, and $\sigma(s, t|v)$ is the number of
	those paths passing through some node $v$ other than $s, t$.
	If $s = t$, $\sigma(s, t) = 1$, and if $v \in {s, t}$,
	$\sigma(s, t|v) = 0$ [2]_.

	Parameters
	----------
	G : graph
	A NetworkX graph.

	k : int, optional (default=None)
	If k is not None use k node samples to estimate betweenness.
	The value of k <= n where n is the number of nodes in the graph.
	Higher values give better approximation.

	normalized : bool, optional
	If True the betweenness values are normalized by `2/((n-1)(n-2))`
	for graphs, and `1/((n-1)(n-2))` for directed graphs where `n`
	is the number of nodes in G.

	weight : None or string, optional (default=None)
	If None, all edge weights are considered equal.
	Otherwise holds the name of the edge attribute used as weight.

	endpoints : bool, optional
	If True include the endpoints in the shortest path counts.

	Returns
	-------
	nodes : dictionary
	Dictionary of nodes with betweenness centrality as the value.


	Notes
	-----
	The algorithm is from Ulrik Brandes [1]_.
	See [4]_ for the original first published version and [2]_ for details on
	algorithms for variations and related metrics.

	For approximate betweenness calculations set k=#samples to use
	k nodes ("pivots") to estimate the betweenness values. For an estimate
	of the number of pivots needed see [3]_.

	For weighted graphs the edge weights must be greater than zero.
	Zero edge weights can produce an infinite number of equal length
	paths between pairs of nodes.


	"""
	betweenness = dict.fromkeys(G, 0.0) # b[v]=0 for v in G
	if k is None:
		nodes = G
	else:
		random.seed(seed)
		nodes = random.sample(G.nodes(), k)
	for s in nodes:

		# single source shortest paths
		if weight is None: # use BFS
			S, P, sigma = _single_source_shortest_path_basic(G, s)
		else: # use Dijkstra's algorithm
			S, P, sigma = _single_source_dijkstra_path_basic(G, s, weight)

		# accumulation
		if endpoints:
			betweenness = _accumulate_endpoints(betweenness, S, P, sigma, s)
		else:
			betweenness = _accumulate_basic(betweenness, S, P, sigma, s)

	# rescaling
	betweenness = _rescale(betweenness, len(G), normalized=normalized,
						directed=G.is_directed(), k=k)
	return betweenness


In [ ]:
G=nx.erdos_renyi_graph(25,0.5)
b=nx.betweenness_centrality(G)
print(b)


{0: 0.005399753225840183, 1: 0.018898392267957486, 2: 0.01853747098312316, 3: 0.015457499424890732, 4: 0.021116574649183345, 5: 0.0018598249576510446, 6: 0.015170075496162455, 7: 0.04027045820524082, 8: 0.01798331499418456, 9: 0.033092219777002385, 10: 0.026571787713092057, 11: 0.05203422784944524, 12: 0.051164857686596815, 13: 0.022210066503544762, 14: 0.050120642240207465, 15: 0.04145613846700803, 16: 0.0026094678268591314, 17: 0.012969015686406989, 18: 0.015135347744043395, 19: 0.01868324911803173, 20: 0.016504721124286338, 21: 0.019036116862203814, 22: 0.02015801227757749, 23: 0.013562293181858401, 24: 0.007969486230355795}


In [ ]:
# G = nx.Graph()  # Buat graf baru

# # Tambahkan simpul dan sambungan antar simpul
# G.add_nodes_from([1, 2, 3, 4])
# G.add_edges_from([(1, 2), (2, 3), (3, 4)])

# betweenness_centrality = nx.betweenness_centrality(G)

# for user, centrality_value in betweenness_centrality.items():
#     print("User:", user, "Betweenness Centrality:", centrality_value)

In [ ]:
# #sort best influence from the highest to the lowest based on betweenness centrality value

# for w in sorted(b, key=b.get, reverse=False):
#    print(w, b[w])

In [ ]:
plt.figure(figsize=(100,100))
nx.draw_networkx(G)
plt.show()